<a href="https://colab.research.google.com/github/No-Qubit-Left-Behind/Control-Engineering-in-TF/blob/master/TF_Propagator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propagator

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import files
%tensorflow_version 2.x
#!pip install tensorflow==2.0.0-beta0
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [4]:
class Propagator(object):
  def __init__(self):
    self.length = 1372707
    self.steps_power = int(np.floor(np.log2(self.length)))
    self.bool_array = self.gen_bool_array()
    self.dim = 2
    x = -0.0602715189469 * tf.constant(
        [[0 + 0j, 0 + 1j], [0 + 1j, 0 + 0j]], dtype=tf.complex128
    )
    y = -0.0602715189469 * tf.constant(
        [[0, 1], [-1, 0]], dtype=tf.complex128
    )
    z = -0.0602715189469 * tf.constant(
        [[0 + 1j, 0 + 0j], [0 + 0j, 0 - 1j]], dtype=tf.complex128
    )
    self.Rabi_strength = tf.Variable(0., dtype=tf.complex128)
    self.resonance_offset = tf.Variable(0., dtype=tf.complex128)
    self.generators =  tf.stack([x, y, z])
    self.controls = tf.convert_to_tensor(
        np.ones((1372707, 2)),
        dtype=tf.complex128
    )

  def gen_bool_array(self):
    length = self.length
    bool_array = []
    for i in range(self.steps_power):
      bool_array.append(bool(np.mod(length, 2)))
      length = np.floor(length / 2)
    return bool_array
  
  @tf.function
  def exponents(self):
    offset_vector = tf.ones(
        (self.length, 1), dtype=tf.complex128
    )
    exponents = tf.linalg.tensordot(
        tf.concat([
                   self.Rabi_strength * self.controls,
                   self.resonance_offset * offset_vector
                   ], axis=1),
        self.generators,
        1
    )
    return exponents
  
  @tf.function
  def exponentials(self):
    exponents = self.exponents()
    return tf.linalg.expm(exponents)
  
  @tf.function
  def propagate_exps(self):
    step_exps = self.exponentials()
    for i in range(self.steps_power):
      if self.bool_array[i]:
        odd_exp = step_exps[-1, :, :]
        step_exps = tf.linalg.matmul(
          step_exps[1::2, :, :],
          step_exps[0:-1, :, :][0::2, :, :]
        )
        step_exps = tf.concat([
            step_exps[0:-1, :, :],
            [tf.linalg.matmul(odd_exp ,step_exps[-1, :, :])]
        ], 0)
      else:
        step_exps = tf.linalg.matmul(
          step_exps[1::2, :, :],
          step_exps[0::2, :, :]
        )
    return tf.squeeze(step_exps)
    
  @tf.function
  def __call__(self):
    z = tf.constant([[1., 0.], [0., -1.]], dtype=tf.complex128)
    propagator = self.propagate_exps()
    z_times_propagator = tf.linalg.matmul(z, propagator)
    z_times_propagator_adjoint = tf.linalg.matmul(
        z, tf.linalg.adjoint(propagator)
    )
    return 0.5 * tf.math.real(
        tf.linalg.trace(
            tf.linalg.matmul(
                z_times_propagator,
                z_times_propagator_adjoint
            )
        )
    )

propagator = Propagator()
propagator.resonance_offset.assign(-0.001)
propagator.Rabi_strength.assign(0.5)
propagator.propagate_exps().numpy()

array([[ 0.99457468-0.00014711j,  0.07355674+0.07355674j],
       [-0.07355674+0.07355674j,  0.99457468+0.00014711j]])